In [1]:
import os
import compas

from compas.datastructures  import Mesh
from compas_view2.app import App

'''
# folder location
dirname = '/content/drive/My Drive/Colab Notebooks'


# 1. load blocks from Tutorial 7
#    Assignment : Load the blocks from your shell
blocks_in_name = '07_blocks_flat_top.json'
blocks_in_path = os.path.join(dirname, blocks_in_name)

blocks = [block for block in compas.json_load(blocks_in_path)]
'''

# TEMPORARY LOAD
blocks = [block for block in compas.json_load("07_blocks_flat_top_wirecutting.json")]

# 2. visualise the blocks
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

for block in blocks:
    viewer.add(block)
viewer.show()

In [2]:
import os
import compas

from compas.datastructures  import Mesh
from compas_view2.app import App

my_block = blocks[0]

# 1. visualise individual block
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block)
viewer.show()

In [3]:
import os
import compas

from compas.datastructures  import Mesh
from compas.geometry  import Vector
from compas_view2.app import App

faces = list(my_block.faces())

# 1. find top face
top = sorted(my_block.faces(), key=lambda face: Vector(* my_block.face_normal(face, unitized=True)).dot([0,0,1]))[0]
my_block.face_attribute(top, 'top', True)

bottom = sorted(my_block.faces(), key=lambda face: Vector(* my_block.face_normal(face, unitized=True)).dot([0,0,1]))[-1]
my_block.face_attribute(bottom, 'bottom', True)

# 5. visualise individual block with top face different color
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block, facecolors={top: (255,0,0), bottom: (0,128,0)})
viewer.show()

In [4]:
import os
import compas

from compas.datastructures  import Mesh
from compas.geometry import bestfit_frame_numpy
from compas.geometry  import Frame, Rotation, Transformation, Plane
from compas.geometry  import Box
from compas_view2.app import App

# 1. get a list of the top faces
top = list(my_block.faces_where({'top': True}))[0]

# 2. get corner vertex coordinates the top faces
corners = my_block.face_coordinates(top)

# 3. generate a bestfit frame of the corners
frame = Frame(*bestfit_frame_numpy(corners))

# 4. generate a world frame
world = Frame.worldXY()

# 5. build the frame to frame transformation
X = Transformation.from_frame_to_frame(frame, world)

# 6. perform the transformation on the block
transformed_block = my_block.transformed(X)

# 7. flip the block to make it oriented for the wirecutting
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]

X2 = Rotation.from_axis_and_angle(yaxis, 3.14159)
rotated_block = transformed_block.transformed(X2)

# 7. set new variable name for the block
final_block = rotated_block


# 9. generate the machining workspace at worldXY
machine_dim = [24.00, 12.00, 15.50]
machine_space = Box(world,machine_dim[0],machine_dim[1],machine_dim[2])


# 8. visualize the correctly oriented block within the machining workspace
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block, facecolors={top: (255,0,0), bottom: (0,128,0)}, opacity=0.5)
viewer.add(final_block, facecolors={top: (255,0,0), bottom: (0,128,0)})

# viewer.add(machine_space, opacity=0.2)

viewer.show()

In [5]:
import os
import compas

from compas.geometry import Frame, Box
from compas.geometry import Scale
from compas.datastructures import Mesh
from compas.geometry import oriented_bounding_box_numpy
from compas_view2.app import App


# 1. compute the bounding box of the block mesh
#    and convert it into a box geometry object

bbox = final_block.vertices_attributes('xyz', keys=final_block.vertices())
box = oriented_bounding_box_numpy(bbox)
blank = Box.from_bounding_box(box)
blank_unsized = Box.from_bounding_box(box)
bbf = blank.frame

# 2. add padding to blank material Box object by scaling up in every direction
#    use the frame of the blank as the origin for scaling up

blank.transform(Scale.from_factors([1.10, 1.10, 1.10], frame=bbf))


# 3. visualize the correctly oriented block within the machining workspace
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(blank, opacity=0.5)
viewer.add(final_block)
viewer.show()

In [6]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.geometry import Scale
from compas.datastructures import Mesh
from compas.geometry import oriented_bounding_box_numpy
from compas_view2.app import App


for face in final_block.faces():
    if final_block.face_attribute(face,'top') == True:
        id_top = face
    if final_block.face_attribute(face,'bottom') == True:
        id_bot = face

bottom = my_block.face_vertices(0)
top = my_block.face_vertices(1)[::-1]

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

for edge in final_block.edges():
    if edge[0] == bottom[0] and edge[1] == top[0]:
        a, b = final_block.edge_coordinates(*edge)
        line = Line(a, b)
        viewer.add(line, linecolor=(0,1.0,0))
        wire_edge = edge


viewer.add(final_block)
viewer.show()

In [13]:
import os
import compas

from compas.geometry import Frame, Box, Line, Point
from compas.datastructures import Mesh

from compas.geometry import intersection_line_plane, bestfit_plane_numpy
from compas_view2.app import App

side_edges = final_block.edge_strip(wire_edge)

top_vertices = blank.top
bot_vertices = blank.bottom
all_vertices = blank.vertices

blank_top = []
for i in top_vertices:
    blank_top.append(all_vertices[i])
    
blank_bot = []
for i in bot_vertices:
    blank_bot.append(all_vertices[i])

# 3. generate a bestfit plane of the corners
plane_top = Plane(*bestfit_plane_numpy(blank_top))
plane_bot = Plane(*bestfit_plane_numpy(blank_bot))

wires = []
for edge in side_edges:
    a, b = final_block.edge_coordinates(*edge)
    line = Line(a, b)

    pt_a = Point(* intersection_line_plane(line, plane_top))
    pt_b = Point(* intersection_line_plane(line, plane_bot))
    wires.append((pt_a,pt_b))


viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.add(plane, opacity=0.4)
viewer.add(blank, opacity=0.3)

for a, b in wires:
    viewer.add(Line(a, b), linewidth=10, color=(1, 0, 0))

viewer.show()
    

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

...

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

In [8]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.datastructures import Mesh

from compas.utilities import pairwise, linspace
from compas_view2.app import App

A, B = zip(*wires)

interpolation = []

# Zip together pairs of points on cycle A and cycle B.

for (a, aa), (b, bb) in zip(pairwise(A), pairwise(B)):

    # For each vector between pairs of points
    a_aa = aa - a
    b_bb = bb - b

    # Identify the length of the longest of the two
    # compute the approximate number of steps required to move 0.01 units at a time.

    l = max(a_aa.length, b_bb.length)
    n = int(l / 0.05)

    for i in linspace(0, 1, num=n):
        if i == 0:
            continue

        ai = a + a_aa * i
        bi = b + b_bb * i

        interpolation.append((ai, bi))
    

interpolation.reverse()

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.add(blank, opacity=0.3)

@viewer.on(interval=10, frames=len(interpolation))
def cut(i):
    a, b = interpolation[i]
    viewer.add(Line(a, b), color=(1, 0, 0))

viewer.show()

In [9]:
import os
import compas

from compas.geometry import Frame, Box, Line, Point, Plane, Scale
from compas.geometry import bestfit_plane_numpy, intersection_plane_plane
from compas.datastructures import Mesh
from compas_view2.app import App

# 1. get a list of the bottom faces
bottom = list(final_block.faces_where({'bottom': True}))[0]

# 2. get corner vertex coordinates the top faces
corners = final_block.face_coordinates(bottom)
print(corners)

# 3. generate a bestfit plane of the corners
plane = Plane(*bestfit_plane_numpy(corners))

left_vertices = blank.left
right_vertices = blank.right
all_vertices = blank.vertices

blank_left = []
for i in left_vertices:
    blank_left.append(all_vertices[i])

# 3. generate a bestfit plane of the corners
left_plane = Plane(*bestfit_plane_numpy(blank_left))

blank_right = []
for i in right_vertices:
    blank_right.append(all_vertices[i])

# 3. generate a bestfit plane of the corners
right_plane = Plane(*bestfit_plane_numpy(blank_right))

inter = intersection_plane_plane(plane, left_plane)
int_line = Line(inter[0], inter[1])

inter_r = intersection_plane_plane(plane, right_plane)
int_line_r = Line(inter_r[0], inter_r[1])

front_vertices = blank.front
back_vertices = blank.back

blank_front = []
for i in front_vertices:
    blank_front.append(all_vertices[i])

blank_back = []
for i in back_vertices:
    blank_back.append(all_vertices[i])

front_plane = Plane(*bestfit_plane_numpy(blank_front))
back_plane = Plane(*bestfit_plane_numpy(blank_back))

top_wire_start = Point(* intersection_line_plane(int_line, front_plane))
top_wire_end = Point(* intersection_line_plane(int_line, back_plane))

top_wire_start2 = Point(* intersection_line_plane(int_line_r, front_plane))
top_wire_end2 = Point(* intersection_line_plane(int_line_r, back_plane))

top_wires = []
top_wires.append((top_wire_start,top_wire_end))
top_wires.append((top_wire_start2,top_wire_end2))

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.add(plane)
viewer.add(int_line, linewidth=10, color=(1, 0, 0))
viewer.add(top_wire_start)
viewer.add(top_wire_end)
viewer.add(top_wire_start2)
viewer.add(top_wire_end2)
viewer.add(blank, opacity=0.4)
viewer.show()

[[1.0982117199372055, -0.6130464048908904, 2.4266404425561743], [0.00015069757852001768, -1.1135281453953292, 2.423365777290946], [-1.0980867411922346, -0.6133065011520715, 2.426642189201121], [-1.089543318003327, 0.4600676510671411, 2.4160260705690404], [-0.0002226906212819434, 1.0484061871348924, 2.3724766001355633], [1.089299595968888, 0.4603569992417986, 2.416000756027195]]


In [10]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.datastructures import Mesh

from compas.utilities import pairwise, linspace
from compas_view2.app import App

A, B = zip(*top_wires)

interpolation = []

# Zip together pairs of points on cycle A and cycle B.

for (a, aa), (b, bb) in zip(pairwise(A), pairwise(B)):

    # For each vector between pairs of points
    a_aa = aa - a
    b_bb = bb - b

    # Identify the length of the longest of the two
    # compute the approximate number of steps required to move 0.01 units at a time.

    l = max(a_aa.length, b_bb.length)
    n = int(l / 0.05)

    for i in linspace(0, 1, num=n):
        if i == 0:
            continue

        ai = a + a_aa * i
        bi = b + b_bb * i

        interpolation.append((ai, bi))
    

interpolation.reverse()

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.add(blank, opacity=0.3)

@viewer.on(interval=10, frames=len(interpolation))
def cut(i):
    a, b = interpolation[i]
    viewer.add(Line(a, b), color=(1, 0, 0))

viewer.show()

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

...

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range

IndexError: list index out of range